In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive/Emotion Recognition')

In [ ]:
df = pd.read_csv('text_emotion.csv')
print(df.shape)
df.head()

(40000, 4)


,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [ ]:
df.describe()

,tweet_id
count,4.000000e+04
mean,1.845184e+09
std,1.188579e+08
min,1.693956e+09
25%,1.751431e+09
50%,1.855443e+09
75%,1.962781e+09
max,1.966441e+09


### Analyzing different types of emotions present in the dataset

### Correction of Mispelled words and slangs

In [ ]:
correction_df = pd.read_csv("Spelling Corrector/aspell.txt",sep=":",names=["correction","misspell"])
correction_df.tail()

,correction,misspell
445,wimp,whimp
446,won't,wan't
447,writing,writting
448,yield,yeild
449,your,youe


In [ ]:
correction_df.drop_duplicates("misspell",inplace=True)

In [ ]:
print(correction_df.shape)

(447, 2)


Mapping the incorrect words to the correct ones

In [ ]:
corr_dict = dict(zip(correction_df.misspell, correction_df.correction))
#Sample of mapping
print(dict(list(corr_dict.items())[0:12]))

{' nevade': 'Nevada', ' presbyterian': 'Presbyterian', ' rsx': 'RSX', ' Steffen': 'Stephen', ' susan': 'Susan', ' abilitey': 'ability', ' abouy': 'about', ' absorbtion': 'absorption', ' accidently': 'accidentally', ' accomodate acommadate': 'accommodate', ' acord': 'accord', ' aquantance': 'acquaintance'}


In [ ]:
#Converting short forms to correct words
short = pd.read_csv("Spelling Corrector/contractions.csv")
short_dict = dict(zip(short["Contraction"], short["Meaning"]))
print(dict(list(short_dict.items())[0:12]))

{"'aight": 'alright', "ain't": 'is not', "amn't": 'am not', "aren't": 'are not', "can't": 'cannot', "'cause": 'because', "could've": 'could have', "couldn't": 'could not', "couldn't've": 'could not have', "daren't": 'dare not', "daresn't": 'dare not', "dasn't": 'dare not'}


In [ ]:
#Functions to correct the tweets
import re
def spell_correction(tweet):
    for word in tweet.split():
        if word in corr_dict.keys():
            tweet = tweet.replace(word,corr_dict[word])
    return tweet
def full_form(tweet):
    for word in tweet.split():
        if word in short_dict.keys():
            tweet = tweet.replace(word,short_dict[word])
    return tweet    
def clean_punctuation(tweet):
    for word in tweet.lower():
        if word in '''()-[]{};:"\,<>./@#$%^&_~''':
            tweet = tweet.replace(word," ")
        elif word in "'":
            tweet = tweet.replace(word,"")
    return tweet

In [ ]:
!pip install emoji
import emoji
import spacy
import en_core_web_sm
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

def clean(df):
        lemmatizer = WordNetLemmatizer()
        stemmer = PorterStemmer()
        df['clean_tweet'] = df['content']
        clean_tokens = []
        stop_words = stopwords.words('english') 
        df['clean_tweet'] = df['clean_tweet'].str.replace("((www\.[^\s]+)|(https?://[^\s]+))","")
        df['clean_tweet'] = df['clean_tweet'].str.replace("@[\w]*","")
        df['clean_tweet'] = df['clean_tweet'].str.replace("[^a-zA-Z0-9' ]"," ")
        df['clean_tweet'] = df['clean_tweet'].str.replace("(^| ).( |$)"," ")
        df['clean_tweet'] = df['clean_tweet'].str.replace("^(RT)","")
        df['clean_tweet'] = df['clean_tweet'].apply(lambda x:spell_correction(x))
        df['clean_tweet'] = df['clean_tweet'].apply(lambda x:full_form(x))
        df['clean_tweet'] = df['clean_tweet'].apply(lambda x:' '.join(clean_punctuation(emoji.demojize(x)).split()))
        return df

In [ ]:
df = clean(df)

In [ ]:
df['clean_tweet'].tail()

39995                                                     
39996                        Happy Mothers Day All my love
39997    Happy Mothers Day to all the mommies out there...
39998    WASSUP BEAUTIFUL FOLLOW ME PEEP OUT MY NEW HIT...
39999    bullet train from tokyo the gf and have been v...
Name: clean_tweet, dtype: object

Drop empty tweets

In [ ]:
print("Before : ",df.shape)
df = df[df.clean_tweet != ""]
print("After : ",df.shape)

Before :  (40000, 5)
After :  (39902, 5)


In [ ]:
df['sentiment'].value_counts()

neutral       8567
worry         8454
happiness     5208
sadness       5162
love          3841
surprise      2185
fun           1776
relief        1526
hate          1323
empty          812
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

### Encoding the sentiments

In [ ]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
sent_to_id  = {"empty":0, "sadness":1,"enthusiasm":2,"neutral":3,"worry":4,"surprise":5,"love":6,"fun":7,"hate":8,"happiness":9,"boredom":10,"relief":11,"anger":12}
df["sentiment_id"] = df['sentiment'].map(sent_to_id)
le = LabelEncoder()
labels = le.fit_transform(df['sentiment_id'])
labels = labels.reshape(len(labels),1)
print(labels)

[[0]
 [1]
 [1]
 ...
 [6]
 [9]
 [6]]


In [ ]:
ohe = OneHotEncoder(sparse=False)
y = ohe.fit_transform(labels)
print(y)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df['clean_tweet'],y,test_size=0.35,random_state=1,shuffle=True)

In [ ]:
x_test.head()

22022                   got to play Off the Wall first tho
12114    are you on about BGT im not watching it tonigh...
1316     Waah You were in Aberdeen why do miss all the ...
31182           my gut feel says its going to be boy ababa
27659                                      Hey Dave whasup
Name: clean_tweet, dtype: object

### Creating word Embeddings

In [ ]:
#!pip install keras
#!pip install tensorflow
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Flatten,Embedding,LSTM,Dropout,GRU

In [ ]:
from keras.layers import Input, Embedding, SpatialDropout1D, LSTM
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers import Bidirectional, Conv1D, Dense, concatenate
from keras.models import Model

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

In [ ]:
vocab_size = len(tokenizer.word_index)+1
train_sequence = tokenizer.texts_to_sequences(x_train)
test_sequence = tokenizer.texts_to_sequences(x_test)

In [ ]:
max_len = max(len(seq) for seq in train_sequence)
print(max_len)

33


In [ ]:
train_pad = pad_sequences(train_sequence,maxlen= max_len,padding='pre')
test_pad = pad_sequences(test_sequence,maxlen= max_len,padding='pre')

In [ ]:
train_pad[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    4, 2136],
      dtype=int32)

### Model Architecture

#### 1. Using Glove Embedding

In [ ]:
import numpy as np
word2vec = {}
vocab = set()
with open('glove.6B.200d.txt',"r",encoding="utf8") as file:
    for line in file:
        word = line.strip().split()
        vocab.add(word[0])
        word2vec[word[0]] =  np.array(word[1:],dtype=float)

In [ ]:
print("Words in Vocabulary : ",len(vocab))

Words in Vocabulary :  400000


In [ ]:
embed_matrix = np.zeros((vocab_size,250))
for word,index in tokenizer.word_index.items():
    embed_vec = tokenizer.word_index.get(word)
    if embed_vec is not None:
        embed_matrix[index] = embed_vec

In [ ]:
len(embed_matrix)

23571

In [ ]:
test_pad.shape[1]

33

#### LSTM 

In [ ]:
model = Sequential()
embed = Embedding(vocab_size,250,input_length = train_pad.shape[1],weights=[embed_matrix],trainable=False)
model.add(embed)
model.add(Dropout(0.2))
model.add(LSTM(250,dropout=0.2,recurrent_dropout=0.2))
# model.add(LSTM(250,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(13,activation='softmax'))
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 33, 250)           5892750   
_________________________________________________________________
dropout (Dropout)            (None, 33, 250)           0         
_________________________________________________________________
gru (GRU)                    (None, 250)               376500    
_________________________________________________________________
dense (Dense)                (None, 13)                3263      
Total params: 6,272,513
Trainable params: 379,763
Non-trainable params: 5,892,750
_________________________________________________________________


In [ ]:
model.fit(train_pad,y_train,epochs=15,batch_size=128,validation_data=(test_pad,y_test))

Epoch 1/15
203/203 [==============================] - 23s 115ms/step - loss: 2.2132 - accuracy: 0.2175 - val_loss: 2.1751 - val_accuracy: 0.2129
Epoch 2/15
203/203 [==============================] - 22s 110ms/step - loss: 2.1558 - accuracy: 0.2244 - val_loss: 2.1648 - val_accuracy: 0.2180
Epoch 3/15
203/203 [==============================] - 22s 108ms/step - loss: 2.1513 - accuracy: 0.2269 - val_loss: 2.1760 - val_accuracy: 0.2119
Epoch 4/15
203/203 [==============================] - 22s 110ms/step - loss: 2.1526 - accuracy: 0.2274 - val_loss: 2.1554 - val_accuracy: 0.2296
Epoch 5/15
203/203 [==============================] - 22s 110ms/step - loss: 2.1477 - accuracy: 0.2291 - val_loss: 2.1622 - val_accuracy: 0.2250
Epoch 6/15
203/203 [==============================] - 23s 111ms/step - loss: 2.1508 - accuracy: 0.2252 - val_loss: 2.1527 - val_accuracy: 0.2268
Epoch 7/15
203/203 [==============================] - 23s 112ms/step - loss: 2.1472 - accuracy: 0.2297 - val_loss: 2.1580 - val_ac

In [ ]:
def get_sentiment(model,text):
    text = spell_correction(text)
    text = full_form(text)
    twt = tokenizer.texts_to_sequences([text])
    twt = pad_sequences(twt, maxlen=max_len, dtype='int32')
    sentiment = model.predict(twt,batch_size=1,verbose = 2)
    sent = np.round(np.dot(sentiment,100).tolist(),0)[0]
    result = pd.DataFrame([sent_to_id.keys(),sent]).T
    result.columns = ["sentiment","percentage"]
    result=result[result.percentage !=0]
    return result

In [ ]:
result =get_sentiment(model,"The pain my heart feels is just too much for it to bear. Nothing eases this pain. I can’t hold myself back. I really miss you")
print(result)

1/1 - 0s
     sentiment percentage
0        empty          2
1      sadness         10
2   enthusiasm          2
3      neutral         21
4        worry         21
5     surprise          6
6         love         14
7          fun          2
8         hate          3
9    happiness         17
11      relief          2


In [ ]:
input_length = train_pad.shape[1]
input_dim = vocab_size
num_classes = 13
embedding_dim = 250
lstm_units = 128
lstm_dropout = 0.1
recurrent_dropout = 0.1
spatial_dropout=0.2
filters=64
kernel_size=3

In [ ]:
input_layer = Input(shape=(input_length,))
output_layer = Embedding(input_dim=vocab_size,output_dim=embedding_dim,input_shape=(input_length,))(input_layer)
output_layer = SpatialDropout1D(spatial_dropout)(output_layer)
output_layer = Bidirectional(LSTM(units=lstm_units,return_sequences=True,dropout=lstm_dropout,recurrent_dropout=recurrent_dropout))(output_layer)
output_layer = Conv1D(filters,kernel_size=kernel_size,kernel_initializer='glorot_uniform')(output_layer)
avgPool = GlobalAveragePooling1D()(output_layer)
maxPool = GlobalMaxPooling1D()(output_layer)
output_layer = concatenate([avgPool,maxPool])
output_layer = Dense(num_classes,activation='softmax')(output_layer)
model = Model(input_layer,output_layer)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
batch_size = 64
epochs = 10

In [ ]:
model.fit(train_pad,y_train,batch_size=batch_size,epochs=epochs)

Epoch 1/10
406/406 [==============================] - 118s 291ms/step - loss: 2.0131 - accuracy: 0.3049
Epoch 2/10
406/406 [==============================] - 119s 292ms/step - loss: 1.7430 - accuracy: 0.4118
Epoch 3/10
406/406 [==============================] - 118s 290ms/step - loss: 1.4284 - accuracy: 0.5257
Epoch 4/10
406/406 [==============================] - 118s 291ms/step - loss: 1.0602 - accuracy: 0.6578
Epoch 5/10
406/406 [==============================] - 118s 291ms/step - loss: 0.7898 - accuracy: 0.7455
Epoch 6/10
406/406 [==============================] - 117s 289ms/step - loss: 0.6109 - accuracy: 0.7980
Epoch 7/10
406/406 [==============================] - 118s 290ms/step - loss: 0.5013 - accuracy: 0.8347
Epoch 8/10
406/406 [==============================] - 118s 290ms/step - loss: 0.4337 - accuracy: 0.8570
Epoch 9/10
406/406 [==============================] - 119s 292ms/step - loss: 0.3793 - accuracy: 0.8729
Epoch 10/10
406/406 [==============================] - 118s 291m

In [ ]:
result = get_sentiment(model,"I was very very upset")
print(result)

1/1 - 0s
    sentiment percentage
1     sadness         22
3     neutral          4
4       worry         48
5    surprise          1
6        love         16
8        hate          1
9   happiness          4
11     relief          2
